## ANN Regressor

An ANN regressor is a type of artificial neural network (ANN) that is used for regression tasks.

ANN is composed of layers of interconnected "neurons," which process and transmit information through the network. Each neuron receives input from the previous layer, processes it using an activation function, and then passes it on to the next layer. The output layer of the ANN regressor produces a prediction for the desired output value, in this case the prediction is on the Sale price of Ames housing database.

The weights and biases of the neurons are adjusted during training, using an optimization algorithm such as Adam.
The goal of training is to find a set of weights and biases that minimizes the error between the predicted output and the true output values in the training data.

Once trained, the ANN regressor can be used to make predictions on new input data by passing it through the network and using the trained weights and biases to calculate the predicted output value.


In [2]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import pandas as pd
from tensorflow_addons.metrics import RSquare
import tensorflow as tf

X_train = pd.read_csv('x_train_preprocessed_minmax.csv')
X_test = pd.read_csv('x_test_preprocessed_minmax.csv')
y_train = pd.read_csv('y_train_preprocessed_minmax.csv')
y_test = pd.read_csv('y_test_preprocessed_minmax.csv')

y_train = y_train.to_numpy().flatten() #y_train flattened, works better

oh_neighbor = []
for col in X_train.columns:
    if 'Neighborhood_b' in col:
        oh_neighbor.append(col)

X_train.drop(columns=oh_neighbor, inplace=True)
X_test.drop(columns=oh_neighbor, inplace=True)

porch = ['Wood_Deck_SF', 'Open_Porch_SF', 'Enclosed_Porch', 'Three_season_porch', 'Screen_Porch']
surface = ['Total_Finished_Bsmt_SF', 'First_Flr_SF', 'Second_Flr_SF', 'Garage_Area']
baths = ['Full_Bath', 'Half_Bath', 'Bsmt_Full_Bath', 'Bsmt_Half_Bath']

X_train.drop(columns=porch, inplace=True)
X_test.drop(columns=porch, inplace=True)

X_train.drop(columns=surface, inplace=True)
X_test.drop(columns=surface, inplace=True)

X_train.drop(columns=baths, inplace=True)
X_test.drop(columns=baths, inplace=True)

All the feature which have been encoded are removed, this solution significatively reduces the loss parameter.

## Network initialization and setup

Using single hidden layer with half of the units of the input one.
The output layer has a single units due to the necessity to predict only the Sale Price.

For the input and hidden layers the activation function chosen is relu because of its good speed of train and simplicity to compute.
The output has instead the linear activation function for predicting the float value.

The kernel initializer is always normal, so it use a normal distribution to generate tensors.
- Parameters of layers
    1) units = number of neurons
    2) kernel_initializer = define the distribution used to inizialize the layer
    3) activation = activation is a function that is applied to the output do decide the use or not of the neuron
    4) input_shape = reflect the number of feature in the dataset
- Parameters of optimizer
    1) learning_rate = step size at which the algorithm makes update to the model / how fast the model learn , slower learner requires more epoch
    2) beta_1 = momentum term in Adam algorithm, larger value more enphasis on the past
    3) beta_2 = decay rate

In [3]:
model = models.Sequential()

model.add(layers.Dense(units=189, kernel_initializer='normal', activation='relu', input_shape=[188]))
model.add(layers.Dense(units=95, kernel_initializer='normal', activation='relu'))
model.add(layers.Dense(units=1, kernel_initializer='normal', activation=None)) #last layer should be linear

opt = optimizers.Adam(learning_rate=0.0015, beta_1=0.9, beta_2=0.999)

## Model evaluation

In [4]:
model.compile(loss='mean_squared_logarithmic_error', optimizer=opt, metrics=[RSquare()])
model.fit(X_train, y_train,epochs=140)
loss = model.evaluate(X_test, y_test)
print('Test loss:', loss)

Epoch 1/140
69/69 [==============================] - 1s 1ms/step - loss: 82.7360 - r_square: -5.0963
Epoch 2/140
69/69 [==============================] - 0s 1ms/step - loss: 38.9929 - r_square: -5.0785
Epoch 3/140
69/69 [==============================] - 0s 1ms/step - loss: 27.4377 - r_square: -5.0458
Epoch 4/140
69/69 [==============================] - 0s 1ms/step - loss: 21.2424 - r_square: -5.0007
Epoch 5/140
69/69 [==============================] - 0s 1ms/step - loss: 17.1615 - r_square: -4.9438
Epoch 6/140
69/69 [==============================] - 0s 1ms/step - loss: 14.1880 - r_square: -4.8730
Epoch 7/140
69/69 [==============================] - 0s 1ms/step - loss: 11.7730 - r_square: -4.7835
Epoch 8/140
69/69 [==============================] - 0s 1ms/step - loss: 9.7903 - r_square: -4.6748
Epoch 9/140
69/69 [==============================] - 0s 1ms/step - loss: 8.2895 - r_square: -4.5575
Epoch 10/140
69/69 [==============================] - 0s 1ms/step - loss: 7.1331 - r_square: 

## Predicted values

In [5]:
from sklearn.metrics import r2_score
y_pred = model.predict(X_test)
print("Score on test split:",r2_score(y_test, y_pred))

23/23 [==============================] - 0s 680us/step
Score on test split: 0.7254087021660028


# Adding layers | overfitting detector / auto_LR_reduction
1) Add more layer, trying with 5
2) Add overfitting detector that will stop before complete all the epoch if needed
3) Add learning rate optimizer that will reduce the LR automatically
4) It is a general practice to reduce the amount of neurons to half of the previous layer

In [6]:
model = models.Sequential()

overfitting_detector = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                        min_delta=0,# minimum change to qualify an improvement
                                                        patience=3, #number of epoch after training will be stopped if we have no improvement
                                                        verbose=1,
                                                        mode='auto', # let algorithm decide when to stop based on quantity monitored
                                                        baseline=None, # we decide to not use any threshold for stopping
                                                        restore_best_weights=False) #restore last model from the best quality epoch previosly found

adjsutable_learning_rate =tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                               factor=0.4, # factor that decide the reduction of LR
                                                               patience=3, #number of epoch after LR will be reduced
                                                               verbose=1,
                                                               mode="auto",
                                                               min_delta=0.0001, # generic used parameter with this particular value (default)
                                                               cooldown=0, # make reduce LR always active
                                                               min_lr=0) # decide if specify a lower bound to the LR

model.add(layers.Dense(units=189, kernel_initializer='normal', activation='relu', input_shape=[188]))
model.add(layers.Dense(units=95, kernel_initializer='normal', activation='relu'))
model.add(layers.Dense(units=47, kernel_initializer='normal', activation='relu'))
model.add(layers.Dense(units=28, kernel_initializer='normal', activation='relu'))
model.add(layers.Dense(units=1, kernel_initializer='normal', activation=None))

# Increment learning rate because we have a way to stop before the end of all epochs = overfitting detector
# Also the learning rate will be automatically reduced by auto_LR_reduction
opt = optimizers.Adam(learning_rate=0.008,beta_1=0.9, beta_2=0.999)


model.compile(loss='mean_squared_logarithmic_error', optimizer=opt, metrics=[RSquare()])
model.fit(X_train, y_train,epochs=500,
          callbacks=[overfitting_detector,adjsutable_learning_rate],
          workers=50, # number of thread parallel if cpu have all used, it will start use GPU
          validation_data=(X_test,y_test)) #split used for test the model

loss = model.evaluate(X_test, y_test)

Epoch 1/500
69/69 [==============================] - 1s 3ms/step - loss: 22.6260 - r_square: -3.9084 - val_loss: 0.9163 - val_r_square: -1.8984 - lr: 0.0080
Epoch 2/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3811 - r_square: -0.7654 - val_loss: 0.1257 - val_r_square: 0.0626 - lr: 0.0080
Epoch 3/500
69/69 [==============================] - 0s 2ms/step - loss: 0.1132 - r_square: 0.1794 - val_loss: 0.0940 - val_r_square: 0.3379 - lr: 0.0080
Epoch 4/500
69/69 [==============================] - 0s 2ms/step - loss: 0.0973 - r_square: 0.3046 - val_loss: 0.0836 - val_r_square: 0.4040 - lr: 0.0080
Epoch 5/500
69/69 [==============================] - 0s 2ms/step - loss: 0.0866 - r_square: 0.3626 - val_loss: 0.0730 - val_r_square: 0.4647 - lr: 0.0080
Epoch 6/500
69/69 [==============================] - 0s 2ms/step - loss: 0.0758 - r_square: 0.4239 - val_loss: 0.0626 - val_r_square: 0.5192 - lr: 0.0080
Epoch 7/500
69/69 [==============================] - 0s 2ms/step - loss:

Note how the new model is improved in velocity and score thanks to auto_LR_reduction and overfitting detector

# Adding more layer with common half neurons technique

In [7]:
model = models.Sequential()

overfitting_detector = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                        min_delta=0,# minimum change to qualify an improvement
                                                        patience=3, #number of epoch after training will be stopped if we have no improvement
                                                        verbose=1,
                                                        mode='auto', # let algorithm decide when to stop based on quantity monitored
                                                        baseline=None, # we decide to not use any threshold for stopping
                                                        restore_best_weights=False) #restore last model from the best quality epoch previosly found

adjsutable_learning_rate =tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                               factor=0.4, # factor that decide the reduction of LR
                                                               patience=3, #number of epoch after LR will be reduced
                                                               verbose=1,
                                                               mode="auto",
                                                               min_delta=0.0001, # generic used parameter with this particular value (default)
                                                               cooldown=0, # make reduce LR always active
                                                               min_lr=0) # decide if specify a lower bound to the LR

model.add(layers.Dense(units=189, kernel_initializer='normal', activation='relu', input_shape=[188]))
model.add(layers.Dense(units=95, kernel_initializer='normal', activation='relu'))
model.add(layers.Dense(units=47, kernel_initializer='normal', activation='relu'))
model.add(layers.Dense(units=28, kernel_initializer='normal', activation='relu'))
model.add(layers.Dense(units=14, kernel_initializer='normal', activation='relu'))
model.add(layers.Dense(units=7, kernel_initializer='normal', activation='relu'))
model.add(layers.Dense(units=3, kernel_initializer='normal', activation='relu'))
model.add(layers.Dense(units=1, kernel_initializer='normal', activation=None))

# Increment learning rate because we have a way to stop before the end of all epochs = overfitting detector
# Also the learning rate will be automatically reduced by auto_LR_reduction
opt = optimizers.Adam(learning_rate=0.008,beta_1=0.9, beta_2=0.999)


model.compile(loss='mean_squared_logarithmic_error', optimizer=opt, metrics=[RSquare()])
model.fit(X_train, y_train,epochs=500,
          callbacks=[overfitting_detector,adjsutable_learning_rate],
          workers=50, # number of thread parallel if cpu have all used, it will start use GPU
          validation_data=(X_test,y_test)) #split used for test the model

loss = model.evaluate(X_test, y_test)
print('Test loss:', loss)

Epoch 1/500
69/69 [==============================] - 1s 4ms/step - loss: 35.6864 - r_square: -3.3858 - val_loss: 0.1775 - val_r_square: -0.2045 - lr: 0.0080
Epoch 2/500
69/69 [==============================] - 0s 2ms/step - loss: 0.1271 - r_square: 0.1166 - val_loss: 0.1071 - val_r_square: 0.2793 - lr: 0.0080
Epoch 3/500
69/69 [==============================] - 0s 2ms/step - loss: 0.1127 - r_square: 0.2305 - val_loss: 0.1012 - val_r_square: 0.3176 - lr: 0.0080
Epoch 4/500
69/69 [==============================] - 0s 2ms/step - loss: 0.1062 - r_square: 0.2634 - val_loss: 0.0945 - val_r_square: 0.3528 - lr: 0.0080
Epoch 5/500
69/69 [==============================] - 0s 2ms/step - loss: 0.0992 - r_square: 0.2938 - val_loss: 0.0872 - val_r_square: 0.3826 - lr: 0.0080
Epoch 6/500
69/69 [==============================] - 0s 2ms/step - loss: 0.0921 - r_square: 0.3340 - val_loss: 0.0806 - val_r_square: 0.4296 - lr: 0.0080
Epoch 7/500
69/69 [==============================] - 0s 2ms/step - loss: 

Slight improvement both on the train and test score based on R2.